In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [10]:
#Trip Date Selection
trip_start = dt.date(2019,9,20)
trip_end = dt.date(2019,10,1)

In [11]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Calculate the date 1 year ago from the last data point in the database
mostrecent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
mostrecent_date

('2017-08-23',)

In [12]:
# Perform a query to retrieve the data and precipitation scores
latestyear = dt.date(2017,8, 23) - dt.timedelta(days = 365)
precipitation = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > latestyear).\
order_by(Measurement.date).all()
precipitation[0]

('2016-08-24', 0.08)

In [13]:
# Save the query results as a Pandas DataFrame 
precipitation_df = pd.DataFrame(precipitation)
precipitation_df.head()

,0,1
0,2016-08-24,0.08
1,2016-08-24,2.15
2,2016-08-24,2.28
3,2016-08-24,NaN
4,2016-08-24,1.22


In [14]:
# Set the index to the date column and Sort the dataframe by date
precipitation_df.set_index('date')
precipitation_df = precipitation_df.sort_index(ascending = True)
precipitation_df.head()

KeyError: "None of ['date'] are in the columns"

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
precipitation_df.plot('date','prcp')
plt.title('Prepicipation Analysis from August 2016 to August 2017')
plt.xlabel('Date')
plt.ylabel('Precipition')
plt.xticks(rotation = 60)
plt.savefig("Resources/Precipitation.png")

# Precipitation

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation_df.describe()

# Design

In [ ]:
# Design a query to show how many stations are available in this dataset?
Stations = session.query(Station).count()
Stations

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
Active_Stations = (session.query(Measurement.station, func.count(Measurement.station)).\
                   group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all())
Most_Active = Active_Stations[0][0]

print("The station with the highest number of obsevations was " + Most_Active + " with " + str(Active_Stations[0][1]) + " observations.")
print("Below is a list of stations and observation counts in descending order.")
for j, i in Active_Stations:
    print("The station " + j + " had " + str(i) + " observations.")

In [ ]:
# Design a query to retrieve the last 12 months of temperature observation data (tobs)
# Filter by the station with the highest number of observations.
temp = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
    filter(Measurement.station == Most_Active).\
    filter(Measurement.date > latestyear).order_by(Measurement.date).all()
df = pd.DataFrame(temp)
df

In [ ]:
#Plot the results as a histogram
df.set_index('date')
df.plot.hist(bins = 12)
plt.title("Station Analysis from August 2016 to August 2017")
plt.xlabel("Temperature")
plt.ylabel("Observations")
plt.savefig("Resources/Station.png")